### 1. calculate the density of each unit in terms of population instead of area.
### 2. Analyze the correlation between every two elements in a total of 60 months from 2015 to 2019.
### 3. Compute and compare the distribution of correlation coefficient of each pair of elements in New York City.

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm
import re
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [2]:
Service_dict={'Illegal Parking':1,'Blocked Driveway':1,'Mass Gathering Complaint':1,'Drug Activity':2,'Hazardous Materials':2,'Smoking':2,'Abandoned Vehicle':3,'Street Sign - Damaged':3,'Noise - Residential':4,'Noise - Street/Sidewalk':4,'Noise - Vehicle':4,'Noise - Commercial':4,'Noise - Park':4,'Noise - Helicopter':4,'Street Sign - Damaged':5,'Animal-Abuse':5,'Maintenance or Facility':5,'Sidewalk Condition':5,'Root/Sewer/Sidewalk Condition':5,'Street Condition':5,'Curb Condition':5,'Homeless Person Assistance':6,'Bus Stop Shelter Placement':6,'Non-Emergency Police Matter':6, "Graffiti":7, 'VIOLATION':8, 'MISDEMEANOR':8}
disordertypes = list(Service_dict.keys())

#### disorder dataset from 311 and NYPD Complaint

In [3]:
data311 = pd.read_csv("../Data/311_Service_Requests_from_2010_to_Present.csv", iterator = True, chunksize=100000,  low_memory=False)
data311

In [4]:
data311Num = 0
data311_useful = []

In [5]:
for chunk in tqdm(data311):
    chunk = chunk[["Incident Zip", "Complaint Type", "Created Date"]]
    chunk = chunk[chunk["Complaint Type"].isin(disordertypes)]
    chunk["year"] = chunk["Created Date"].apply(lambda x: int(x.split(" ")[0].split("/")[-1]))
    chunk = chunk[(chunk.year>=2015)&(chunk.year<=2019)]
    data311_useful.append(chunk)
    chunkNum = len(chunk)
    data311Num += chunkNum
#     if data311Num >= 1000000:
#         break

252it [08:11,  1.95s/it]


In [6]:
final311data = pd.concat(data311_useful,axis=0,ignore_index=True)
final311data.head()

,Incident Zip,Complaint Type,Created Date,year
0,11228,Maintenance or Facility,11/06/2017 10:10:20 AM,2017
1,10014,Maintenance or Facility,11/06/2017 12:09:17 PM,2017
2,11238,Maintenance or Facility,11/06/2017 03:09:48 PM,2017
3,10306,Sidewalk Condition,11/06/2017 11:35:52 AM,2017
4,11416,Blocked Driveway,11/06/2017 12:59:47 PM,2017


In [7]:
final311data["date"] = final311data["Created Date"].apply(lambda x:x.split(" ")[0])
final311data["month"] = final311data["Created Date"].apply(lambda x:int(x.split(" ")[0].split("/")[0]))
final311data["timestamp"] = final311data["Created Date"].apply(lambda x:x.split(" ")[1])
final311data["AM_PM"] = final311data["Created Date"].apply(lambda x:x.split(" ")[2])
final311data["timeinterval"] = final311data["timestamp"].apply(lambda x: int(x.split(":")[0]))
final311data["AM_PM"] = final311data["AM_PM"].apply(lambda x: 0 if x=="AM" else 12)
final311data["timeinterval"] = final311data["timeinterval"] + final311data["AM_PM"]
final311data["timeinterval"] = final311data["timeinterval"].apply(lambda x: x%24)
final311data.rename(columns={"Incident Zip":"zipcode", "Complaint Type":"type"}, inplace=True)
final311data = final311data[['zipcode', 'date', 'year', 'month', 'timeinterval', 'type']]

In [8]:
final311data.head()

,zipcode,date,year,month,timeinterval,type
0,11228,11/06/2017,2017,11,10,Maintenance or Facility
1,10014,11/06/2017,2017,11,0,Maintenance or Facility
2,11238,11/06/2017,2017,11,15,Maintenance or Facility
3,10306,11/06/2017,2017,11,11,Sidewalk Condition
4,11416,11/06/2017,2017,11,0,Blocked Driveway


#### NYPD Complaint

In [9]:
NYPDComplaint = pd.read_csv("../Data/NYPD_Complaint_Data_Historic.csv",  iterator = True, chunksize=100000,  low_memory=False)
NYPDComplaint

In [10]:
dataNYPD_useful = []

In [11]:
for chunk in tqdm(NYPDComplaint):
    chunk = chunk[['X_COORD_CD', 'Y_COORD_CD', "LAW_CAT_CD", "CMPLNT_FR_DT", "CMPLNT_FR_TM"]]
    chunk.dropna(axis=0,how='any', inplace=True)
    chunk = chunk[chunk["LAW_CAT_CD"].isin(disordertypes)]
    chunk["year"] = chunk["CMPLNT_FR_DT"].apply(lambda x: int(x.split("/")[-1]))
    chunk = chunk[(chunk.year>=2015)&(chunk.year<=2019)]
    dataNYPD_useful.append(chunk)

70it [01:19,  1.13s/it]


In [12]:
finalNYPDdata = pd.concat(dataNYPD_useful,axis=0,ignore_index=True)
finalNYPDdata["X_COORD_CD"] = finalNYPDdata["X_COORD_CD"].apply(lambda x:int("".join(list(filter(str.isdigit, x)))))
finalNYPDdata["Y_COORD_CD"] = finalNYPDdata["Y_COORD_CD"].apply(lambda x:int("".join(list(filter(str.isdigit, x)))))
finalNYPDdata.head()

,X_COORD_CD,Y_COORD_CD,LAW_CAT_CD,CMPLNT_FR_DT,CMPLNT_FR_TM,year
0,1007522,247458,VIOLATION,05/28/2015,15:00:00,2015
1,1005028,234516,MISDEMEANOR,04/13/2016,00:00:00,2016
2,988958,209044,VIOLATION,06/25/2017,14:00:00,2017
3,999240,169262,MISDEMEANOR,03/26/2017,12:00:00,2017
4,1010854,182099,VIOLATION,04/27/2015,12:40:00,2015


In [13]:
len(finalNYPDdata)

1623296

#### process NYPD with shapefiles

In [14]:
#load the resulting shapefile using geopandas 
zipcode = gpd.read_file('../Data/ZIPCODE/ZIP_CODE_040114.shp')
zipcode.head()

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,"POLYGON ((1038098.252 188138.380, 1038141.936 ..."
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1001613.713 186926.440, 1002314.243 ..."
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1011174.276 183696.338, 1011373.584 ..."
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((995908.365 183617.613, 996522.848 18..."
4,11218,0,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((991997.113 176307.496, 992042.798 17..."


In [15]:
zipcode_polygon = zipcode[["ZIPCODE", "geometry"]].values
def getZipcode(p):
    for code, geometry in zipcode_polygon:
        if geometry.contains(p):
            return int(code)

In [16]:
zipcode_list = []
for idx, row in tqdm(finalNYPDdata.iterrows()):
    X_COORD_CD, Y_COORD_CD, LAW_CAT_CD, CMPLNT_FR_DT, CMPLNT_FR_TM, year = row
    code = getZipcode(Point(X_COORD_CD, Y_COORD_CD))
    zipcode_list.append(code)

1623296it [26:48, 1009.01it/s]


In [17]:
finalNYPDdata["zipcode"] = zipcode_list

In [18]:
finalNYPDdata = finalNYPDdata[["zipcode", "year", "CMPLNT_FR_DT", "LAW_CAT_CD", "CMPLNT_FR_TM"]]
finalNYPDdata["month"] = finalNYPDdata["CMPLNT_FR_DT"].apply(lambda x:int(x.split("/")[0]))
finalNYPDdata["timeinterval"] = finalNYPDdata["CMPLNT_FR_TM"].apply(lambda x:int(x.split(":")[0]))
finalNYPDdata.rename(columns={"CMPLNT_FR_DT":"date", "LAW_CAT_CD":"type"}, inplace=True)
finalNYPDdata = finalNYPDdata[["zipcode", "date", "year", "month", "timeinterval", "type"]]
finalNYPDdata.head()

,zipcode,date,year,month,timeinterval,type
0,10452.0,05/28/2015,2015,5,15,VIOLATION
1,10454.0,04/13/2016,2016,4,0,MISDEMEANOR
2,10010.0,06/25/2017,2017,6,14,VIOLATION
3,11210.0,03/26/2017,2017,3,12,MISDEMEANOR
4,11212.0,04/27/2015,2015,4,12,VIOLATION


### merge dataset from two source

In [19]:
final311data.head()

,zipcode,date,year,month,timeinterval,type
0,11228,11/06/2017,2017,11,10,Maintenance or Facility
1,10014,11/06/2017,2017,11,0,Maintenance or Facility
2,11238,11/06/2017,2017,11,15,Maintenance or Facility
3,10306,11/06/2017,2017,11,11,Sidewalk Condition
4,11416,11/06/2017,2017,11,0,Blocked Driveway


In [20]:
finalNYPDdata.head()

,zipcode,date,year,month,timeinterval,type
0,10452.0,05/28/2015,2015,5,15,VIOLATION
1,10454.0,04/13/2016,2016,4,0,MISDEMEANOR
2,10010.0,06/25/2017,2017,6,14,VIOLATION
3,11210.0,03/26/2017,2017,3,12,MISDEMEANOR
4,11212.0,04/27/2015,2015,4,12,VIOLATION


In [21]:
DisorderData = pd.concat([final311data, finalNYPDdata],axis=0,ignore_index=True)
disordertype_name = {1:"Uncivil Use of Space", 2:"Drug & Alcohol", 3:"Trash", 4:"Noise", 5:"Damage", 6:"Others", 7:"Graffiti", 8:"Misdemeanor & Violation"}
DisorderData.rename(columns={"type":"disorder_name"}, inplace=True)
DisorderData["disorder_type"] = DisorderData["disorder_name"].apply(lambda x:Service_dict[x])
DisorderData["disorder_name"] = DisorderData["disorder_type"].apply(lambda x:disordertype_name[x])
DisorderData["dayOfWeek"] = DisorderData["date"].apply(lambda x: datetime.strptime(x,"%m/%d/%Y").weekday() + 1)
DisorderData["isWeekend"] = DisorderData["dayOfWeek"].apply(lambda x:1 if x>5 else 0)
DisorderData = DisorderData[['zipcode', 'date', 'year', 'month', 'timeinterval', 'isWeekend', "disorder_name", "disorder_type"]]
DisorderData.head()

,zipcode,date,year,month,timeinterval,isWeekend,disorder_name,disorder_type
0,11228,11/06/2017,2017,11,10,0,Damage,5
1,10014,11/06/2017,2017,11,0,0,Damage,5
2,11238,11/06/2017,2017,11,15,0,Damage,5
3,10306,11/06/2017,2017,11,11,0,Damage,5
4,11416,11/06/2017,2017,11,0,0,Uncivil Use of Space,1


In [22]:
DisorderData.to_csv("DisorderData.csv", index=None)